### Monday, December 4, 2023

Trying it again ...

pip install exllamav2

docker container start hfpt_Oct28

### Sunday, December 3, 2023

Downloaded the "TheBloke/Python-Code-33B-GPTQ" model yesterday ... ran a quick test this morning. It works!

### Friday, November 17, 2023

https://huggingface.co/TheBloke/Python-Code-33B-GPTQ

In [1]:
# You need this to just target the 4090.
# only target the 4090 ...
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [3]:
model_name_or_path = "TheBloke/Python-Code-33B-GPTQ"

In [4]:
!ls /home/rob/Data2/huggingface/transformers

models--TheBloke--CodeLlama-34B-Instruct-GPTQ  version.txt
models--TheBloke--Python-Code-33B-GPTQ


In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)


In [6]:
%%time
# To use a different branch, change revision
# For example: revision="gptq-4bit-128g-actorder_True"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             # use_exllama_v2=True, ... this does not work
                                             revision="main")


# Load time
# CPU times: user 19.9 s, sys: 9.32 s, total: 29.3 s
# Wall time: 12.4 s
# 18424MiB VRAM

# Download time
# CPU times: user 1min 32s, sys: 1min 39s, total: 3min 12s
# Wall time: 4h 37s

You have activated exllama backend. Note that you can get better inference
                    speed using exllamav2 kernel by setting `use_exllama_v2=True`.`disable_exllama` will be deprecated
                    in future version.


CPU times: user 21.6 s, sys: 7.09 s, total: 28.6 s
Wall time: 10.5 s


In [7]:
# docker cp c9b676310ea0://home/rob/Data2/huggingface/transformers/models--TheBloke--Python-Code-33B-GPTQ /home/rob/Data3/huggingface/transformers
# Successfully copied 16.9GB to /home/rob/Data3/huggingface/transformers

In [8]:
prompt = "Tell me about AI"
prompt_template=f'''This is a conversation with your helpful AI assistant. AI assistant can generate Python Code along with necessary explanation.

Context
You are a helpful AI assistant.

USER: {prompt}
ASSISTANT:
'''

# print("\n\n*** Generate:")

In [9]:
input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()

In [13]:
%%time
maxTokens = 2048
output2048 = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=maxTokens)
output = output2048[0]
print(len(output))
print(tokenizer.decode(output))


# 38.4s
# 20.8s

423
<s> This is a conversation with your helpful AI assistant. AI assistant can generate Python Code along with necessary explanation.

Context
You are a helpful AI assistant.

USER: Tell me about AI
ASSISTANT:

Artificial Intelligence (AI) is a field of computer science that studies intelligent agents. An intelligent agent is a system that perceives its environment and takes actions to maximize its utility. 

AI researchers study intelligent agents using a variety of methods, including mathematical proofs, algorithmic techniques, and computer simulations. The field has made significant progress in the past few decades, and many of the techniques and algorithms used in AI are now widely used in other fields, such as computer vision, machine learning, and data science.

Some popular topics in AI include:

- Machine learning: This subfield of AI studies how agents can learn to behave in an optimal way, given a set of rules or a training dataset.

- Deep learning: This subfield of machine

In [14]:
# Inference can also be done using transformers' pipeline

# print("*** Pipeline:")
tfPipeLine = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)

In [15]:
print(tfPipeLine(prompt_template)[0]['generated_text'])

# 20324 MiB VRAM
# 19906 MiB VRAM

# 15.0s
# 12.9s 
# 15.2s

This is a conversation with your helpful AI assistant. AI assistant can generate Python Code along with necessary explanation.

Context
You are a helpful AI assistant.

USER: Tell me about AI
ASSISTANT:
AI or Artificial Intelligence is a subfield of Computer Science that studies intelligent agents. An intelligent agent is a software program that can perceive its environment, understand the goals and constraints of the problem at hand, and come up with a plan to solve it. 

In the past few decades, AI has made significant progress in various fields such as computer vision, natural language processing, and machine learning. For example, AI assistants like Siri and Alexa have become commonplace in many households, able to perform tasks like scheduling appointments, playing music, and answering basic questions. 

However, there is still a long way to go before AI can match the complexity and nuance of human intelligence. Many researchers and engineers are working on solving problems like l

So let's try this with [LangChain](https://python.langchain.com/docs/integrations/llms/huggingface_pipelines), shall we ... 

In [16]:
from langchain.llms.huggingface_pipeline import HuggingFacePipeline as LCHuggingFacePipeLine

In [17]:
# pass in the transformers pipeline we defined above ... 
lchfpl = LCHuggingFacePipeLine(pipeline=tfPipeLine)

Below is the replicated code from a previous cell ... 

In [21]:
prompt = "Tell me about AI"

prompt_template=f'''This is a conversation with your helpful AI assistant. AI assistant can generate Python Code along with necessary explanation.

Context
You are a helpful AI assistant.

USER: {prompt}
ASSISTANT:
'''


In [18]:
from langchain.prompts import PromptTemplate as LCPromptTemplate

lcPromptTemplate = LCPromptTemplate.from_template(prompt_template)

chain = lcPromptTemplate | lchfpl

In [20]:

result = chain.invoke({"prompt": prompt})

In [21]:
print(result)

Artificial Intelligence (AI) is a field of computer science that studies intelligent agents. An intelligent agent is a program or system that can learn, reason, and act in an intelligent manner. 

AI researchers study how to make machines smart, using techniques such as machine learning, pattern recognition, and optimization. The goal of AI is to create systems that can solve problems as well as or better than humans, in a faster and more efficient way. 

AI has many applications, including in the fields of robotics, drones, self-driving cars, and natural language processing. It is also used in software development, where it can be used to automate tasks and improve code quality. 

Desp the term "artificial" in Artificial Intelligence, AI systems are not truly intelligent in the sense that they are not conscious or aware of their surroundings. Instead, they are designed to simulate intelligence by using advanced algorithms and computational power. 

AI technology is rapidly improving, 

In [23]:
type(result)

str